# Import librairies

In [1]:
import preprocessing.preprocessing as pp
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau



nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rokra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rokra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rokra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rokra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rokra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rokra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load data, variations

In [10]:
df = pp.load_data("english_cleaned_lyrics.csv")
df_no_lemmatize_keep_stop = pp.preprocess_data(df.copy(), lemmatize=False, remove_stops=False)
df_no_lemmatize = pp.preprocess_data(df.copy(), lemmatize=False)
df_keep_stop = pp.preprocess_data(df.copy(), remove_stops=False)
df = pp.preprocess_data(df.copy())
df = pp.adjust_genre_distribution(df)
df_no_lemmatize_keep_stop = pp.adjust_genre_distribution(df_no_lemmatize)
df_no_lemmatize = pp.adjust_genre_distribution(df_no_lemmatize)
df_keep_stop = pp.adjust_genre_distribution(df_keep_stop)
df = df[['lyrics', 'genre']]
df


,lyrics,genre
0,oh baby know im gonna cut right chase woman ma...,Pop
1,playin everything easy like seem sure still wa...,Pop
2,search tenderness isnt hard find love need liv...,Pop
3,oh oh oh oh oh oh wrote book stand title book ...,Pop
4,party people people party popping sitting arou...,Pop
...,...,...
218205,gotta say boy couple date youre hand outright ...,Country/Folk
218206,helped find diamond ring made try everything t...,Country/Folk
218207,look couple corner booth look lot like shes lo...,Country/Folk
218208,fly mortal earth im measured depth girth fathe...,Country/Folk


In [11]:
X = df['lyrics'].to_list()
y = df['genre'].to_list()

# Encode

In [12]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

In [13]:
max_words = 25000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train_seq = pad_sequences(sequences, maxlen=max_len)

test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_seq = pad_sequences(test_sequences, maxlen=max_len)

# Create model

In [14]:
model = Sequential()
model.add(Embedding(input_dim=max_words + 1, output_dim=20))
model.add(LSTM(15, dropout=0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Train

In [15]:
history = model.fit(X_train_seq, y_train, batch_size=32, epochs=5, validation_split=0.2)

Epoch 1/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 118s 40ms/step - accuracy: 0.3776 - loss: 1.4917 - val_accuracy: 0.4977 - val_loss: 1.2644
Epoch 2/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 124s 42ms/step - accuracy: 0.4936 - loss: 1.2715 - val_accuracy: 0.5205 - val_loss: 1.2275
Epoch 3/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 186s 63ms/step - accuracy: 0.5324 - loss: 1.2037 - val_accuracy: 0.5199 - val_loss: 1.2270
Epoch 4/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 113s 38ms/step - accuracy: 0.5502 - loss: 1.1645 - val_accuracy: 0.5375 - val_loss: 1.1955
Epoch 5/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 103s 35ms/step - accuracy: 0.5681 - loss: 1.1290 - val_accuracy: 0.5355 - val_loss: 1.1998


In [16]:
loss, accuracy = model.evaluate(X_test_seq, y_test)
print("Test Accuracy:", accuracy)

916/916 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.5353 - loss: 1.1959
Test Accuracy: 0.5297049880027771


# Skeleton to try with different datas

In [17]:
def skeleton_model(df):
    X = df['lyrics'].to_list()
    y = df['genre'].to_list()
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)

    X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)
    max_words = 25000
    max_len = 200

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)
    sequences = tokenizer.texts_to_sequences(X_train)
    X_train_seq = pad_sequences(sequences, maxlen=max_len)

    test_sequences = tokenizer.texts_to_sequences(X_test)
    X_test_seq = pad_sequences(test_sequences, maxlen=max_len)
    model = Sequential()
    model.add(Embedding(input_dim=max_words + 1, output_dim=20))
    model.add(LSTM(15, dropout=0.5))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_seq, y_train, batch_size=32, epochs=5, validation_split=0.2)
    loss, accuracy = model.evaluate(X_test_seq, y_test)
    print("Test Accuracy:", accuracy)

# Best model no lemmatize and keep stopwords

In [18]:
skeleton_model(df_no_lemmatize_keep_stop)

Epoch 1/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 152s 51ms/step - accuracy: 0.3730 - loss: 1.5190 - val_accuracy: 0.4875 - val_loss: 1.2943
Epoch 2/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 151s 51ms/step - accuracy: 0.4963 - loss: 1.2667 - val_accuracy: 0.5169 - val_loss: 1.2349
Epoch 3/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 150s 51ms/step - accuracy: 0.5300 - loss: 1.2019 - val_accuracy: 0.5245 - val_loss: 1.2192
Epoch 4/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 118s 40ms/step - accuracy: 0.5576 - loss: 1.1469 - val_accuracy: 0.5313 - val_loss: 1.2228
Epoch 5/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 107s 36ms/step - accuracy: 0.5738 - loss: 1.1132 - val_accuracy: 0.5369 - val_loss: 1.2033
916/916 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.5385 - loss: 1.1952
Test Accuracy: 0.538036048412323


# Best model keep stopwords

In [19]:
skeleton_model(df_keep_stop)

Epoch 1/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 111s 37ms/step - accuracy: 0.3266 - loss: 1.5927 - val_accuracy: 0.4279 - val_loss: 1.4301
Epoch 2/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 108s 37ms/step - accuracy: 0.4179 - loss: 1.4334 - val_accuracy: 0.4371 - val_loss: 1.3863
Epoch 3/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 118s 40ms/step - accuracy: 0.4622 - loss: 1.3386 - val_accuracy: 0.4581 - val_loss: 1.3142
Epoch 4/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 107s 36ms/step - accuracy: 0.4710 - loss: 1.2965 - val_accuracy: 0.4765 - val_loss: 1.2807
Epoch 5/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 112s 38ms/step - accuracy: 0.5005 - loss: 1.2404 - val_accuracy: 0.4896 - val_loss: 1.2802
916/916 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.4896 - loss: 1.2819
Test Accuracy: 0.49009832739830017


# Best model no lemmatize

In [20]:
skeleton_model(df_no_lemmatize)

Epoch 1/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 112s 37ms/step - accuracy: 0.3797 - loss: 1.5049 - val_accuracy: 0.4700 - val_loss: 1.3362
Epoch 2/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 103s 35ms/step - accuracy: 0.4772 - loss: 1.3176 - val_accuracy: 0.4675 - val_loss: 1.3234
Epoch 3/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 109s 37ms/step - accuracy: 0.5101 - loss: 1.2473 - val_accuracy: 0.5095 - val_loss: 1.2604
Epoch 4/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 107s 37ms/step - accuracy: 0.5392 - loss: 1.1953 - val_accuracy: 0.5196 - val_loss: 1.2420
Epoch 5/5
2929/2929 ━━━━━━━━━━━━━━━━━━━━ 120s 41ms/step - accuracy: 0.5497 - loss: 1.1731 - val_accuracy: 0.5260 - val_loss: 1.2225
916/916 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.5245 - loss: 1.2152
Test Accuracy: 0.5255394577980042
